In [1]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from mne.stats import permutation_cluster_test
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization

### Add Directories/ Load Files

In [2]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


### Select Peaks and Plot Analytic Signal

In [46]:
#Load raw fif data

test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'test',
        'Sub045_FIF.fif'
    )
)

subID = 'Sub045'
fft_name = str(subID) + '_'
print(fft_name)

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub045_FIF.fif...
    Reading extended channel information
    Range : 0 ... 145611 =      0.000 ...   582.444 secs
Ready.
Sub045_


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_14180\2581033189.py:3: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\test\Sub045_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  test_raw = mne.io.read_raw_fif(os.path.join(


In [47]:
reload(dat_preproc)
x = test_raw.get_data() 
x1 = x[0,:]

peakMed = 82
peakStim = 63

dat_ngam = dat_preproc.low_highpass_filter(x1, peakMed-2, peakMed+2) 
dat_subh = dat_preproc.low_highpass_filter(x1, peakStim-2, peakStim+2) 

datall = [dat_ngam, dat_subh] 
labels = ['Peak'+str(peakMed)+'Hz','Peak'+str(peakStim)+'Hz']
print(labels)

['Peak82Hz', 'Peak63Hz']


In [48]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [49]:
%matplotlib qt
sm_signal_np = np.empty(shape = (2, x1.shape[0] - 499))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(1, 1, figsize=(12, 5))
wintosmooth = 500

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(np.squeeze(amplitude_envelope))

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    plt.plot(sm_signal, label = labels[idx])
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    #axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    #axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    plt.ylabel('Analytic Signal')
    plt.xlim([0, sm_signal.shape[0]])

    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 500 samples')
plt.legend(loc='upper right')

plt.show()


In [50]:
x2 = x[4, :] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim)

In [52]:
154*250

38500

In [66]:
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,:], label = labels[idx], lw = 2)
ax2.plot(sm_stim1, label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
#ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
#ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
#ax1.set_xlim(0, sm_signal_np.shape[0])
#ax1.set_xticks(np.arange(0, 100000, 20000))
#ax1.set_xticklabels(np.arange(0, 400, 80))
ax1.set_xlabel('Time [samples]')
plt.title(str(subID))
plt.show()

In [53]:
sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]], [sm_signal_np[1]],[sm_stim1]])))
sm_analSignal_df = pd.DataFrame(sm_analSignal, 
    columns = ['Spontan','StimOn','StimVec'],
    )
print(sm_analSignal_df)

         Spontan    StimOn  StimVec
0       1.368847  0.790434      0.0
1       1.369139  0.787469      0.0
2       1.369938  0.785298      0.0
3       1.370787  0.783349      0.0
4       1.371630  0.781279      0.0
...          ...       ...      ...
145108  1.512171  1.147411      NaN
145109  1.513409  1.147936      NaN
145110  1.514641  1.148590      NaN
145111  1.515861  1.149501      NaN
145112  1.517489  1.150105      NaN

[145113 rows x 3 columns]


In [21]:
plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignal',dpi = 300)

In [18]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','anal_signal/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignal',dpi = 300)
sm_analSignal_df.to_csv(str(fft_file)+str(fft_name)+'sm_analSignal.csv')

NameError: name 'sm_analSignal_df' is not defined

### Crop Analytic Signal to Epochs of Interest

In [5]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'Sub006_sm_analSignal.csv'
))

print(sm_anal)

subID = 'Sub006'
fft_name = str(subID) + '_'
print(fft_name)

        Unnamed: 0   Spontan    StimOn  StimVec
0                0  1.635867  0.881777      0.0
1                1  1.635703  0.879257      0.0
2                2  1.635288  0.876840      0.0
3                3  1.634423  0.874098      0.0
4                4  1.633745  0.871046      0.0
...            ...       ...       ...      ...
288809      288809  0.965999  0.926363      NaN
288810      288810  0.968719  0.928732      NaN
288811      288811  0.971397  0.930926      NaN
288812      288812  0.973054  0.933220      NaN
288813      288813  0.975737  0.935677      NaN

[288814 rows x 4 columns]
Sub006_


In [14]:
300000/250

1200.0

In [13]:
np.arange(0,1400,200)

array([   0,  200,  400,  600,  800, 1000, 1200])

In [20]:
%matplotlib qt
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
ax1.plot(np.arange(0, sm_anal.shape[0]), sm_anal['Spontan'], label = 'Peak 81Hz')
ax1.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimOn'], label = 'Peak 65Hz')
ax2.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimVec'], ls='--', color = 'grey')

ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax1.set_xticklabels(np.arange(-200,1400,200))
ax1.set_xlabel('Time [sec]')


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_5564\2428986198.py:12: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(np.arange(-200,1400,200))


Text(0.5, 0, 'Time [sec]')

In [61]:
20*250

5000

In [67]:
#CROP IT

med1 = 52
med2 = 82
sub_pt = 36600
time_on = sub_pt - 25*250
time_off = sub_pt + 15*250

sm_anal1 = sm_anal.iloc[med1*250 : med2*250]
sm_anal2 = sm_anal.iloc[time_on : time_off]
sm_toplot = pd.concat([sm_anal1, sm_anal2])
print(sm_toplot)

       Unnamed: 0   Spontan    StimOn   StimVec
13000       13000  1.102678  1.133032  0.000000
13001       13001  1.088468  1.132112  0.000000
13002       13002  1.074449  1.131211  0.000000
13003       13003  1.060656  1.130331  0.000000
13004       13004  1.047124  1.129470  0.000000
...           ...       ...       ...       ...
40345       40345  1.235583  0.906546  1.741172
40346       40346  1.236496  0.904866  1.742177
40347       40347  1.237407  0.903261  1.743181
40348       40348  1.238313  0.901729  1.744185
40349       40349  1.239211  0.900269  1.745188

[17500 rows x 4 columns]


In [69]:
# CROP ONLY THE TRANSITION E.G. BETWEEN 0.5 - 1.0mA
this_sig = sm_toplot

fig, ax1 = plt.subplots(figsize = (8,6))
ax2 = ax1.twinx()

ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['Spontan'], label = 'Med-Induced')
ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['StimOn'], label = 'Subharmonic')
ax2.plot(np.arange(0, this_sig.shape[0]), this_sig['StimVec'], ls='--', color = 'grey')
ax1.axvline(x = 13750, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
ax1.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)


ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax1.set_xlabel('Time [samples]')
ax1.set_title(str(subID))


Text(0.5, 1.0, 'Sub045')

In [70]:
fft_fig = os.path.join(project_path, 'figures','anal_signal/')
fft_file = os.path.join(project_path, 'data','anal_signal/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalTRANSITION',dpi = 150)
this_sig.to_csv(str(fft_file)+str(fft_name)+'sm_analSignalTRANSITION.csv')

### Average Cropped Analytic Signal and Plot

In [13]:
'''
med_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllMed_analSignalTRANSITION.csv'
))

stim_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllStim_analSignalTRANSITION.csv'
))

print(stim_anal)
'''

med_anal = []
stim_anal = []

ps_path = os.path.join(
   project_path,
   'data', 'anal_signal'
)

for filename in os.listdir(ps_path):
   if filename.endswith('TRANSITION.csv'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
      this_df = pd.read_csv(os.path.join(ps_path,filename))
      this_med_anal = this_df['Spontan']
      this_stim_anal = this_df['StimOn']


      med_anal.append(this_med_anal)
      stim_anal.append(this_stim_anal)



In [23]:
all_med_anal_df = pd.DataFrame(med_anal)
all_stim_anal_df = pd.DataFrame(stim_anal)
all_med_anal_df.shape

(7, 17500)

In [63]:
med_anal_cr = all_med_anal_df.loc[:, 1:7500] #all_med_anal_df.loc[:, 7500:17500]
med_anal_mean = np.nanmean(med_anal_cr,0)
med_anal_sem = stats.sem(med_anal_cr,0, nan_policy='omit')

stim_anal_cr = all_stim_anal_df.loc[:,  1:7500]
stim_anal_mean = np.nanmean(stim_anal_cr,0)
stim_anal_sem = stats.sem(stim_anal_cr,0, nan_policy='omit')


In [60]:
med_anal_cr


,0,1,2,3,4,5,6,7,8,9,...,7491,7492,7493,7494,7495,7496,7497,7498,7499,7500
Spontan,1.390779,1.391188,1.391186,1.390794,1.390600,1.390491,1.390341,1.390117,1.389762,1.389299,...,1.283606,1.285311,1.287010,1.288701,1.290388,1.292072,1.293753,1.295435,1.297117,0.907698
Spontan,1.635867,1.635703,1.635288,1.634423,1.633745,1.633078,1.632535,1.632077,1.631430,1.630612,...,1.440437,1.441429,1.442537,1.443744,1.445035,1.446395,1.447810,1.449266,1.450749,1.015672
Spontan,0.967818,0.968528,0.968903,0.969024,0.969884,0.971084,0.972555,0.974410,0.976562,0.978987,...,0.913803,0.912992,0.912088,0.911092,0.910005,0.908830,0.907568,0.906224,0.904800,0.941739
Spontan,0.678128,0.679558,0.680983,0.682395,0.683786,0.685145,0.686464,0.687736,0.688951,0.690101,...,0.554113,0.552090,0.550108,0.548170,0.546277,0.544431,0.542634,0.540889,0.539198,1.428828
Spontan,0.819748,0.823732,0.828641,0.833606,0.838637,0.843713,0.848834,0.853991,0.859151,0.864292,...,1.492297,1.494871,1.497429,1.499964,1.502473,1.504950,1.507391,1.509791,1.512146,0.634824
Spontan,1.443568,1.442884,1.442797,1.442759,1.442646,1.442483,1.442224,1.441880,1.441566,1.441273,...,0.972828,0.970471,0.968003,0.965425,0.962739,0.959944,0.957049,0.954190,0.951439,0.948798
Spontan,1.102678,1.088468,1.074449,1.060656,1.047124,1.033887,1.020980,1.008437,0.996293,0.984581,...,1.058753,1.057070,1.055200,1.053151,1.050932,1.048556,1.046034,1.043377,1.040600,0.521533


In [66]:
2000/250

8.0

In [73]:
%matplotlib qt
plt.plot(np.arange(0,7500), med_anal_mean, label = 'Med-Induced FTG') #10000
plt.fill_between(np.arange(0,7500), med_anal_mean-med_anal_sem, med_anal_mean+med_anal_sem, alpha = 0.2)

plt.plot(np.arange(0,7500), stim_anal_mean, color = 'red', label = 'Stim-Induced FTG')
plt.fill_between(np.arange(0,7500), stim_anal_mean-stim_anal_sem, stim_anal_mean+stim_anal_sem, color = 'red', alpha = 0.2)


#plt.axvline(x = 6250, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
#plt.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)
#plt.xlim(0,10000)
#plt.xticks(np.arange(0,11250,1250), labels = np.arange(-25,20,5))
plt.xlim(0,2500)

plt.legend(loc = 'upper left')
plt.xlabel('Time [sec]')
plt.ylabel('Z-scored Smoothed Analytic Signal')

Text(0, 0.5, 'Z-scored Smoothed Analytic Signal')

In [54]:
fft_fig = os.path.join(project_path, 'results\\')

plt.savefig(str(fft_fig)+'AVG_AnalCropped_StimOn',dpi = 250)

In [ ]:
pval = 0.05  # arbitrary
dfn = 2 - 1  # degrees of freedom numerator
dfd = med_anal.shape[0] - 2  # degrees of freedom denominator
thresh = scipy.stats.f.ppf(1 - pval, dfn=dfn, dfd=dfd)  # F distribution
print(thresh)

In [ ]:
perm_nd = np.empty([6,med_anal.shape[0]])
perm_nd[:] = np.nan
perm_nd[0,:] = np.transpose(med_anal)

In [ ]:
y = np.expand_dims(med_anal, axis = 2)
y1 = np.expand_dims(stim_anal, axis = 2)


In [ ]:
#H0 = np.empty([1000,6])
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([y, y1], n_permutations=1000,
                             threshold=None, n_jobs=None)


In [ ]:
cluster_p_values

### Make all Signals same Length

In [3]:
import traces


In [4]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'Sub005_sm_analSignal.csv'
))

print(sm_anal)

subID = 'Sub005'
fft_name = str(subID) + '_'
print(fft_name)

        Unnamed: 0   Spontan    StimOn  StimVec
0                0  1.390779  0.796159      0.0
1                1  1.391188  0.796054      0.0
2                2  1.391186  0.796050      0.0
3                3  1.390794  0.796008      0.0
4                4  1.390600  0.795919      0.0
...            ...       ...       ...      ...
174121      174121  0.675485  0.660545      NaN
174122      174122  0.676904  0.661651      NaN
174123      174123  0.678758  0.662717      NaN
174124      174124  0.680079  0.663879      NaN
174125      174125  0.681471  0.664938      NaN

[174126 rows x 4 columns]
Sub005_


In [30]:
%matplotlib qt
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['Spontan'], label = 'Med Induced')
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimOn'], label = 'Stim Induced')
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimVec'], ls='--', color = 'grey')
plt.legend()

In [29]:
np.unique(np.round(sm_anal['StimVec'], decimals = 1))

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       nan])

In [29]:
sm_anal['StimVec'] = np.round(sm_anal['StimVec'], decimals = 1)
this_anal = sm_anal
plt.plot(sm_anal['Spontan'][sm_anal['StimVec'] == 0])
plt.plot(sm_anal['StimOn'][sm_anal['StimVec'] == 0])
plt.plot(sm_anal['StimVec'][sm_anal['StimVec'] == 0])

In [31]:
amp_of_interest = [0, 0.5, 1]

sm_anal['Spontan'][250:300,sm_anal['StimVec'] == 0]

KeyError: 'key of type tuple not found and not a MultiIndex'